In [8]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: srsly<3.0.0,>=2.4.3 in c:\users\89bal\anaconda3.9\lib\site-packages (from spacy) (2.4.3)



In [9]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('punkt')
import spacy 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\89bal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
tweets = pd.read_csv("Elon_musk.csv")
tweets.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 20: invalid start byte

In [ ]:
tweets.shape

In [ ]:
tweets = tweets.drop(labels='Unnamed: 0',axis=1)
tweets

In [ ]:
tweets = [Text.strip() for Text in tweets.Text]
tweets = [Text for Text in tweets if Text]

In [ ]:
tweets[:7]

In [ ]:
tweets_text = " ".join(tweets)
tweets_text

In [ ]:
from nltk.tokenize import TweetTokenizer

In [ ]:
toknzr = TweetTokenizer(strip_handles=True)
tweet_tokenz = toknzr.tokenize(tweets_text)

In [ ]:
print(tweet_tokenz)

In [ ]:
tweet_tokenz_text = ' '.join(tweet_tokenz)

In [ ]:
tweet_tokenz_text

In [ ]:
no_punc_text = tweet_tokenz_text.translate(str.maketrans('','',string.punctuation))
no_punc_text

In [ ]:
import re
no_url_text = re.sub(r'http\S+','',no_punc_text)
no_url_text

In [ ]:
from nltk.tokenize import word_tokenize
text_tokens = word_tokenize(no_url_text)
print(text_tokens)

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')
ex  = ['’','rt','ye','yeah','haha','Yes','U0001F923','I','…']
my_stopwords.extend(ex)

In [ ]:
no_stop_token = [i for i in text_tokens if not i in my_stopwords] 
print(no_stop_token)

In [ ]:
lower_words = [i.lower() for i in no_stop_token]
print(lower_words[:100])

In [ ]:
!pip3 install en_core_web_sm
!python -m spacy download en

In [ ]:
import spacy
from spacy.lang.en.examples import sentences 

nlp = spacy.load("en_core_web_sm")
doc = nlp(' '.join(lower_words))
print(doc)

In [ ]:
lemmas = [token.lemma_ for token in doc]
print(lemmas)

In [ ]:
tweets_clean = ' '.join(lemmas)
tweets_clean

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(ngram_range=(1, 3),max_features=500)
tfidfv_matrix = tfidfv.fit_transform(lemmas)

In [ ]:
print(tfidfv.get_feature_names())
print(tfidfv_matrix.toarray())

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud,STOPWORDS

In [ ]:
def plot_cloud(wordcloud):
    plt.figure(figsize=(40,30))
    plt.imshow(wordcloud)
    plt.axis('off')
    
STOPWORDS.add('pron')
STOPWORDS.add('rt')
STOPWORDS.add('yeah')
wordcloud=WordCloud(width=3000,height=2000,background_color='black',max_words=50,
                   colormap='Set1',stopwords=STOPWORDS).generate(tweets_clean)
plot_cloud(wordcloud)

In [ ]:
nlp = spacy.load('en_core_web_sm')
one_block = tweets_clean
doc_block = nlp(one_block)
spacy.displacy.render(doc_block,style='ent',jupyter=True)

## SENTIMENT ANALYSIS

In [ ]:
tweets

In [ ]:
from nltk import tokenize
sentnc = tokenize.sent_tokenize(' '.join(tweets))
sentnc

In [ ]:
sent_df = pd.DataFrame(sentnc,columns=['sentance'])

In [ ]:
sent_df

In [12]:
afinn = pd.read_csv("Afinn.csv",encoding='unicode_escape')

In [13]:
afinn

,word,value
0,abandon,-2
1,abandoned,-2
2,abandons,-2
3,abducted,-2
4,abduction,-2
...,...,...
2472,yucky,-2
2473,yummy,3
2474,zealot,-2
2475,zealots,-2


In [ ]:
afinnity_score = afinn.set_index('word')['value'].to_dict()
afinnity_score

In [ ]:
nlp = spacy.load('en_core_web_sm')
sentiment = afinnity_score
def sentiment_cal (text:str=None):
    sent_score = 0
    if text:
        sentance = nlp(text)
        for word in sentance:
            sent_score+=sentiment.get(word.lemma_,0)
    return sent_score        

In [ ]:
sentiment_cal(text='that is awful')

In [ ]:
sent_df['sentiment_value'] = sent_df['sentance'].apply(sentiment_cal)

In [ ]:
sent_df

In [ ]:
sent_df['word_count'] = sent_df['sentance'].str.split().apply(len)
sent_df

In [ ]:
sent_df[sent_df['sentiment_value']<=0]

In [ ]:
sent_df[sent_df['sentiment_value']>0]

In [ ]:
sns.distplot(sent_df["sentiment_value"])

In [ ]:
sent_df.plot.scatter(x='sentiment_value',y='word_count',figsize = (6,6))